In [1]:
# Import dependences
from searchtweets import ResultStream, gen_rule_payload, load_credentials
from searchtweets import collect_results
import pymongo
import time
import argparse
import json
import string
from dateutil import parser as psr
from datetime import datetime
from pprint import pprint
from config import dbname, dbuser, psswd, host, parameters

connection_string ='mongodb+srv://' + dbuser + ':' + psswd + host + '/' + dbname + "?" + parameters
mongo_client = pymongo.MongoClient(connection_string)


In [2]:
premium_search_args = load_credentials(filename="./search_tweets_creds_example.yaml",
                 yaml_key="search_tweets_fullarchive_dev",
                 env_overwrite=False)

Grabbing bearer token from OAUTH


In [3]:
# Create the rules to filter tweets
stringRule = """
(Greta Thunberg OR #greta OR #gretathunberg)
lang:en
"""

# Put the filters params together
rule = gen_rule_payload(stringRule,
                        from_date="2019-09-24 11:00", #UTC 2017-09-01 00:00
                        to_date="2019-09-24 12:00",#UTC 2017-10-30 00:00
#                         from_date="2019-09-23 13:00", #UTC 2017-09-01 00:00
#                         to_date="2019-09-23 23:59",#UTC 2017-10-30 00:00
                        results_per_call=100)
print(rule)

{"query": "(Greta Thunberg OR #greta OR #gretathunberg) lang:en", "maxResults": 100, "toDate": "201909241200", "fromDate": "201909241100"}


In [4]:
# Run the query and get the tweets
tweets = collect_results(rule, max_results=500, result_stream_args=premium_search_args)

In [5]:
# Save the tweets in a json file
with open('.\data\GretaThunberg.json', 'a') as json_file:
    for tweet in tweets:
        json.dump(tweet, json_file)

In [6]:
# Save the tweets on MongoDB
for tweet in tweets:
    try:        
        user = {
                "id":           tweet["user"]["id"],
                "id_str":       tweet["user"]["id_str"], 
                "name":         tweet["user"]["name"],
                "screen_name":  tweet["user"]["screen_name"],
                "location":     tweet["user"]["location"],
                "description":  tweet["user"]["description"],
                "created_at":   psr.parse(tweet["user"]["created_at"])
                }

        tweetObj= {
                    '#tag':"Greta Thunberg",
                    'text': tweet["text"],
                    'module_sent_an': None,
                    'module_pollution': None,
                    'module_politics': None,
                    'module_environment': None,
                    'module_hate': None,
                    'twitter_created_at': psr.parse(tweet["created_at"]),
                    'dbai_created_at': datetime.now(),
                    'user': user,
                    'coordinates': tweet["coordinates"],
                    'place': tweet["place"]
                    }
        mongo_client.dbAI.twitter.insert_one(tweetObj)
    except BaseException as e:
        print("Error db_data: %s" % str(e))    
    

In [7]:
len(tweets)

500